<a href="https://colab.research.google.com/github/iraidaantropova/StrD/blob/main/HW1_SinitsaI_StrD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=0c72f7a521105c0618b090a1cb483c6d0c71e174b7e43700323f4cad1198e79f
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession, DataFrame, functions as F
from pyspark.sql.types import StringType, StructType

In [4]:
#работа с потоками
spark = SparkSession.builder.appName("gogin_spark").getOrCreate()

In [5]:
### Rate sourse
raw_rate = spark \
  .readStream \
  .format('rate') \
  .load()

In [6]:
raw_rate.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- value: long (nullable = true)



In [7]:
raw_rate.isStreaming

True

In [8]:
!pwd

/content


In [9]:
stream = raw_rate.writeStream \
  .trigger(processingTime='30 seconds') \
  .format('csv') \
  .option("path", "/content") \
  .option("checkpointLocation", "/content") \
  .start()

In [10]:
stream.status

{'message': 'Waiting for next trigger',
 'isDataAvailable': True,
 'isTriggerActive': False}

In [11]:
stream.explain()

== Physical Plan ==
*(1) Project [timestamp#0, value#1L]
+- MicroBatchScan[timestamp#0, value#1L] class org.apache.spark.sql.execution.streaming.sources.RateStreamTable$$anon$1




In [12]:
stream.isActive

True

In [13]:
stream.lastProgress

{'id': '3756e0a0-3643-4d3a-8bb7-2e85a1309ea4',
 'runId': 'eaddbd70-264a-424e-9875-adbc947b46ac',
 'name': None,
 'timestamp': '2023-10-10T02:31:30.000Z',
 'batchId': 2,
 'numInputRows': 30,
 'inputRowsPerSecond': 1.0,
 'processedRowsPerSecond': 121.45748987854252,
 'durationMs': {'addBatch': 175,
  'commitOffsets': 34,
  'getBatch': 1,
  'latestOffset': 0,
  'queryPlanning': 11,
  'triggerExecution': 247,
  'walCommit': 20},
 'stateOperators': [],
 'sources': [{'description': 'RateStreamV2[rowsPerSecond=1, rampUpTimeSeconds=0, numPartitions=default',
   'startOffset': 11,
   'endOffset': 41,
   'latestOffset': 41,
   'numInputRows': 30,
   'inputRowsPerSecond': 1.0,
   'processedRowsPerSecond': 121.45748987854252}],
 'sink': {'description': 'FileSink[/content]', 'numOutputRows': -1}}

In [14]:
stream.stop()